In [15]:
import cv2
import os
import numpy as np
from collections import Counter

# 导入训练向量

In [16]:
# 返回的是480张100*100的矩阵，和index
listdir = os.listdir('./face_train')
names = [d for d in listdir if not d.startswith('.')]
#print(names)
images = []
target = []
for index,dir in enumerate(names):
    for i in range(1, 11):
        gray = cv2.imread('./face_train/%s/%d.jpg' % (dir, i))  # 三维图片
        gray_ = gray[:, :, 0]  # 二维数组
        gray_ = cv2.resize(gray_, dsize=(100, 100))
        gray_ = cv2.equalizeHist(gray_) # 图片均衡化处理，颜色鲜明      
        images.append(gray_)
        target.append(index)
images = np.asarray(images)
target = np.asarray(target)

In [17]:
# 转换成矩阵
#图像数据转换特征矩阵
image_data = []

for image in images:
    #转换成一维的数组
    data = image.flatten()
    image_data.append(data)

In [18]:
#转换为numpy数组
X = np.array(image_data)
print(type(X))
print(X.shape)
print(X)

<class 'numpy.ndarray'>
(200, 10000)
[[243 251 245 ... 187 159 199]
 [245 250 249 ... 214 214 214]
 [244 244 244 ... 196 197 205]
 ...
 [212 212 212 ...  79 193 202]
 [191 192 192 ...  35  75 127]
 [211 211 211 ...  14  17  22]]


# 将训练数据进行PCA

In [19]:
# 对数据进行中心化，即每一列减去该列的均值
X_train_centered = X - np.mean(X, axis=0)

In [20]:
# 主成分数量
k = 100
# 对训练矩阵进行SVD
U, S, Vt = np.linalg.svd(X_train_centered, full_matrices=False)
# 选择前100个主成分
X_train_pca = U[:, :k]
print(X_train_pca.shape)
# 构建变换矩阵P
P = np.dot(X.T, X_train_pca)
print(P)

(200, 100)
[[-243.60307749  -70.88445368 -197.19809481 ...  -16.3659918
   -13.29909361    1.29672786]
 [-347.32460601 -148.14210274 -227.81523411 ...    4.02500674
     1.60848943    7.52786466]
 [-461.33838782 -212.21303964 -208.16433207 ...  -17.64214017
    -5.73082261   14.41773545]
 ...
 [-838.01814509 -339.83895176 -292.88521132 ...  -21.76215235
    -1.3250331   -30.11995548]
 [-782.89091494 -389.65077454 -285.87381798 ...  -55.38711864
    51.69244544  -71.23905647]
 [-748.0964684  -378.87127483 -330.67170325 ...  -44.2147517
    12.9159565   -30.4397922 ]]


In [21]:
cap = cv2.VideoCapture(0)
# 人脸检测
face_detector = cv2.CascadeClassifier('E:/tools/opencv/sources/data/haarcascades/haarcascade_frontalface_alt2.xml')
while True:
    flag,frame = cap.read()
    if not flag:
        break
   
    gray = cv2.cvtColor(frame,code = cv2.COLOR_BGR2GRAY)
    
    # 提取测试向量
    faces = face_detector.detectMultiScale(gray, minNeighbors=12)
    for x,y,w,h in faces:       
        
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face,dsize=(100,100))
        #face = cv2.equalizeHist(face)
        
        imgs = []
        imgs.append(face)
        imgs = np.asarray(imgs)
        
        #图像数据转换特征矩阵
        image_data = []

        data = imgs.flatten()
        image_data.append(data)
        
        #转换为numpy数组
        test_vector = np.array(image_data)
        #print(test_vector)

        # 归一化测试向量
        test_vector_centered = test_vector - np.mean(test_vector)
        #print(test_vector_centered)
        
        # 对测试向量PCA
        test_vector_pca = np.dot(test_vector_centered,P)
        
        #print(test_vector_pca)

        # 初始化一个列表来存储每个训练向量与当前测试向量的欧几里得范数
        distances = []

        for j in range(len(X_train_pca)):
            train_vector = X_train_pca[j]

            dist = np.sqrt(np.sum((train_vector-test_vector_pca) ** 2))
            distances.append(dist)

        # 找到最近的k个邻居
        nearest_indices = np.argsort(distances)[:3]
        print(nearest_indices)
        # 找到最频繁的索引
        counts = Counter(nearest_indices//10)
        most_common_index = counts.most_common(1)[0][0]
        
        print(most_common_index)

        # 人脸辨识，返回index和置信度
        y_ = most_common_index
        
        label = names[y_]
        
        print('这个人是：%s。'%(label))
        cv2.rectangle(frame,pt1 = (x,y),pt2 = (x+w,y+h),color=[0,0,255], thickness = 2)
        cv2.putText(frame,text = label, org = (x, y-10),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1.5,
                    color=[0,0,255],
                    thickness=2)
    cv2.imshow('face',frame)
    key = cv2.waitKey(1000//30)
    
    if key == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()


[17 18 19]
1
这个人是：Chen Jiayin。
[11 14 16]
1
这个人是：Chen Jiayin。
[16 14 11]
1
这个人是：Chen Jiayin。
[17 11 16]
1
这个人是：Chen Jiayin。
[14 13 12]
1
这个人是：Chen Jiayin。
[14 13 16]
1
这个人是：Chen Jiayin。
[17 18 19]
1
这个人是：Chen Jiayin。
[11 14 16]
1
这个人是：Chen Jiayin。
[11 14 17]
1
这个人是：Chen Jiayin。
[16 18 17]
1
这个人是：Chen Jiayin。
[14 13 16]
1
这个人是：Chen Jiayin。
[16 14 13]
1
这个人是：Chen Jiayin。
[13 14 12]
1
这个人是：Chen Jiayin。
[17 11 18]
1
这个人是：Chen Jiayin。
[19 17 11]
1
这个人是：Chen Jiayin。
[14 13 11]
1
这个人是：Chen Jiayin。
[19 17 11]
1
这个人是：Chen Jiayin。
[11 19 17]
1
这个人是：Chen Jiayin。
[11 14 17]
1
这个人是：Chen Jiayin。
[11 19 17]
1
这个人是：Chen Jiayin。
[19 17 18]
1
这个人是：Chen Jiayin。
[17 19 18]
1
这个人是：Chen Jiayin。
[19 17 18]
1
这个人是：Chen Jiayin。
[12 11 10]
1
这个人是：Chen Jiayin。
[11 17 19]
1
这个人是：Chen Jiayin。
[11 19 17]
1
这个人是：Chen Jiayin。
[78 76 77]
7
这个人是：Li Xiufang。
[158 154 151]
15
这个人是：Yu Hong。
[11 17 19]
1
这个人是：Chen Jiayin。
[12 13 10]
1
这个人是：Chen Jiayin。
[15 10 12]
1
这个人是：Chen Jiayin。
[11 16 17]
1
这个人是：Chen Jiayin。
[14 13 16

[17 11 16]
1
这个人是：Chen Jiayin。
[17 18 19]
1
这个人是：Chen Jiayin。
[11 17 19]
1
这个人是：Chen Jiayin。
[19 17 11]
1
这个人是：Chen Jiayin。
[11 19 17]
1
这个人是：Chen Jiayin。
[19 17 11]
1
这个人是：Chen Jiayin。
[11 19 17]
1
这个人是：Chen Jiayin。
[19 17 11]
1
这个人是：Chen Jiayin。
[19 17 11]
1
这个人是：Chen Jiayin。
[10 12 11]
1
这个人是：Chen Jiayin。
[14 11 13]
1
这个人是：Chen Jiayin。
[11 14 13]
1
这个人是：Chen Jiayin。
[11 12 10]
1
这个人是：Chen Jiayin。
[11 14 17]
1
这个人是：Chen Jiayin。
[11 19 17]
1
这个人是：Chen Jiayin。
[151 158 154]
15
这个人是：Yu Hong。
[19 11 17]
1
这个人是：Chen Jiayin。
[158 151 154]
15
这个人是：Yu Hong。
[158 156 155]
15
这个人是：Yu Hong。
[158 154 152]
15
这个人是：Yu Hong。
[19 17 18]
1
这个人是：Chen Jiayin。
[19  2 17]
1
这个人是：Chen Jiayin。
[18 19 17]
1
这个人是：Chen Jiayin。
[18 19 17]
1
这个人是：Chen Jiayin。
[18 17 19]
1
这个人是：Chen Jiayin。
[16 13 14]
1
这个人是：Chen Jiayin。
[16 18 14]
1
这个人是：Chen Jiayin。
[18 19 17]
1
这个人是：Chen Jiayin。
[19 18 17]
1
这个人是：Chen Jiayin。
[18 19 17]
1
这个人是：Chen Jiayin。
[ 4 11  5]
0
这个人是：Chen Aijun。
[10 11 12]
1
这个人是：Chen Jiayin。
[11  4  7